# Customer Segmentation & Churn Pattern Analytics in European Banking
### European Bank Dataset | Unified Mentor × European Central Bank
**Student: M. Anjali | Financial Analyst Intern | February 2026**

---
## Step 1: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Plot styling
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

BLUE   = '#1F4E79'
LIGHT  = '#2E75B6'
RED    = '#E74C3C'
GREEN  = '#27AE60'
GOLD   = '#F39C12'

print('All libraries imported successfully!')

---
## Step 2: Load and Explore the Dataset

In [ ]:
df = pd.read_csv('European_Bank__1_.csv')
print(f'Dataset Shape: {df.shape}')
print(f'Rows: {df.shape[0]:,} | Columns: {df.shape[1]}')
df.head()

In [ ]:
print('=== Data Types ===')
print(df.dtypes)
print('\n=== Missing Values ===')
print(df.isnull().sum())
print('\n=== No missing values found — dataset is clean ===')

In [ ]:
df.describe().round(2)

---
## Step 3: Data Cleaning & Preparation

In [ ]:
# Drop non-analytical columns
df.drop(columns=['Surname'], inplace=True)

# Create segmentation bands
df['AgeGroup'] = pd.cut(df['Age'],
                         bins=[0, 30, 45, 60, 100],
                         labels=['Under 30', '30-45', '46-60', 'Over 60'])

df['CreditBand'] = pd.cut(df['CreditScore'],
                           bins=[0, 550, 700, 851],
                           labels=['Low (<550)', 'Medium (550-700)', 'High (>700)'])

df['TenureGroup'] = pd.cut(df['Tenure'],
                            bins=[-1, 2, 5, 10],
                            labels=['New (0-2yr)', 'Mid (3-5yr)', 'Long (6+yr)'])

df['BalanceSeg'] = pd.cut(df['Balance'],
                           bins=[-1, 1, 50000, 300000],
                           labels=['Zero Balance', 'Low (<50k)', 'High (50k+)'])

df['ChurnLabel'] = df['Exited'].map({1: 'Churned', 0: 'Retained'})

print('Segmentation fields created successfully!')
df[['Age','AgeGroup','CreditScore','CreditBand','Tenure','TenureGroup','Balance','BalanceSeg']].head()

---
## Step 4: Overall Churn Rate

In [ ]:
total    = len(df)
churned  = df['Exited'].sum()
retained = total - churned
churn_rate = df['Exited'].mean() * 100

print('=' * 45)
print('       OVERALL CHURN SUMMARY')
print('=' * 45)
print(f'  Total Customers  :  {total:,}')
print(f'  Churned          :  {churned:,}  ({churn_rate:.1f}%)')
print(f'  Retained         :  {retained:,}  ({100-churn_rate:.1f}%)')
print('=' * 45)

# Donut chart
fig, ax = plt.subplots(figsize=(6, 5))
sizes  = [retained, churned]
colors = [GREEN, RED]
wedges, texts, autotexts = ax.pie(
    sizes, colors=colors, autopct='%1.1f%%',
    startangle=90, pctdistance=0.75,
    wedgeprops=dict(width=0.5, edgecolor='white', linewidth=2)
)
for at in autotexts:
    at.set_fontsize(13); at.set_fontweight('bold'); at.set_color('white')
ax.legend(['Retained', 'Churned'], loc='lower center', ncol=2, frameon=False)
ax.set_title('Overall Churn Distribution', fontsize=14, fontweight='bold', color=BLUE)
plt.tight_layout()
plt.show()

---
## Step 5: Churned vs Retained — Profile Comparison

In [ ]:
profile = df.groupby('ChurnLabel')[['Age','Balance','CreditScore','EstimatedSalary']].mean().round(1)
print('=== Average Profile: Churned vs Retained ===')
print(profile.T.to_string())

fig, axes = plt.subplots(1, 4, figsize=(14, 4))
metrics = ['Age', 'Balance', 'CreditScore', 'EstimatedSalary']
labels  = ['Avg Age', 'Avg Balance (€)', 'Avg Credit Score', 'Avg Salary (€)']

for ax, metric, label in zip(axes, metrics, labels):
    vals   = [profile.loc['Retained', metric], profile.loc['Churned', metric]]
    colors = [GREEN, RED]
    bars   = ax.bar(['Retained', 'Churned'], vals, color=colors, edgecolor='white', width=0.5)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()*1.01,
                f'{v:,.0f}', ha='center', fontsize=10, fontweight='bold')
    ax.set_title(label, fontsize=11, fontweight='bold', color=BLUE)
    ax.set_facecolor('#FAFAFA')
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_ylim(0, max(vals)*1.2)

plt.suptitle('Churned vs Retained — Average Profile Comparison', fontsize=13, fontweight='bold', color=BLUE, y=1.02)
plt.tight_layout()
plt.show()

---
## Step 6: Geographic Segmentation

In [ ]:
geo = df.groupby('Geography').agg(
    Customers=('Exited','count'),
    Churned=('Exited','sum'),
    ChurnRate=('Exited','mean'),
    AvgBalance=('Balance','mean'),
    AvgAge=('Age','mean')
).round(2)
geo['ChurnRate%'] = (geo['ChurnRate']*100).round(1)
print(geo[['Customers','Churned','ChurnRate%','AvgBalance','AvgAge']])

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Churn rate bar
colors = [RED if r == geo['ChurnRate%'].max() else LIGHT for r in geo['ChurnRate%']]
bars = axes[0].bar(geo.index, geo['ChurnRate%'], color=colors, edgecolor='white', width=0.5)
for bar, v in zip(bars, geo['ChurnRate%']):
    axes[0].text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.3,
                 f'{v}%', ha='center', fontsize=12, fontweight='bold')
axes[0].set_title('Churn Rate by Country', fontsize=13, fontweight='bold', color=BLUE)
axes[0].set_ylabel('Churn Rate (%)')
axes[0].set_facecolor('#FAFAFA'); axes[0].grid(axis='y', alpha=0.3, linestyle='--')
axes[0].set_ylim(0, 40)

# Stacked bar
x = np.arange(len(geo))
axes[1].bar(x, geo['Customers']-geo['Churned'], label='Retained', color=GREEN, width=0.5)
axes[1].bar(x, geo['Churned'], bottom=geo['Customers']-geo['Churned'], label='Churned', color=RED, width=0.5)
axes[1].set_xticks(x); axes[1].set_xticklabels(geo.index)
axes[1].set_title('Customer Count by Country', fontsize=13, fontweight='bold', color=BLUE)
axes[1].set_ylabel('Number of Customers')
axes[1].legend(); axes[1].set_facecolor('#FAFAFA'); axes[1].grid(axis='y', alpha=0.3, linestyle='--')

plt.suptitle('Geographic Segmentation Analysis', fontsize=14, fontweight='bold', color=BLUE)
plt.tight_layout()
plt.show()

print('\nKey Finding: Germany churns at 32.4% — double France (16.2%) and Spain (16.7%)')

---
## Step 7: Age Segmentation

In [ ]:
age = df.groupby('AgeGroup', observed=True).agg(
    Customers=('Exited','count'),
    Churned=('Exited','sum'),
    ChurnRate=('Exited','mean')
).reset_index()
age['ChurnRate%'] = (age['ChurnRate']*100).round(1)
print(age[['AgeGroup','Customers','Churned','ChurnRate%']])

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

colors = [RED if v == age['ChurnRate%'].max() else LIGHT for v in age['ChurnRate%']]
bars = axes[0].bar(age['AgeGroup'], age['ChurnRate%'], color=colors, edgecolor='white', width=0.6)
for bar, v in zip(bars, age['ChurnRate%']):
    axes[0].text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.5,
                 f'{v}%', ha='center', fontsize=12, fontweight='bold')
axes[0].set_title('Churn Rate by Age Group', fontsize=13, fontweight='bold', color=BLUE)
axes[0].set_ylabel('Churn Rate (%)')
axes[0].set_facecolor('#FAFAFA'); axes[0].grid(axis='y', alpha=0.3, linestyle='--')
axes[0].set_ylim(0, 65)

# Age distribution histogram by churn
axes[1].hist(df[df['Exited']==0]['Age'], bins=30, alpha=0.6, color=GREEN, label='Retained', edgecolor='white')
axes[1].hist(df[df['Exited']==1]['Age'], bins=30, alpha=0.7, color=RED,   label='Churned',  edgecolor='white')
axes[1].set_title('Age Distribution: Churned vs Retained', fontsize=13, fontweight='bold', color=BLUE)
axes[1].set_xlabel('Age'); axes[1].set_ylabel('Number of Customers')
axes[1].legend(); axes[1].set_facecolor('#FAFAFA'); axes[1].grid(axis='y', alpha=0.3, linestyle='--')

plt.suptitle('Age Segmentation Analysis', fontsize=14, fontweight='bold', color=BLUE)
plt.tight_layout()
plt.show()

print('\nKey Finding: The 46-60 age group churns at 51.1% — the highest of any segment')

---
## Step 8: Gender Segmentation

In [ ]:
gen = df.groupby('Gender').agg(
    Customers=('Exited','count'),
    Churned=('Exited','sum'),
    ChurnRate=('Exited','mean')
).reset_index()
gen['ChurnRate%'] = (gen['ChurnRate']*100).round(1)
print(gen[['Gender','Customers','Churned','ChurnRate%']])

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

axes[0].bar(gen['Gender'], gen['ChurnRate%'], color=[RED, LIGHT], edgecolor='white', width=0.5)
for i, (_, row) in enumerate(gen.iterrows()):
    axes[0].text(i, row['ChurnRate%']+0.3, f"{row['ChurnRate%']}%",
                 ha='center', fontsize=13, fontweight='bold')
axes[0].set_title('Churn Rate by Gender', fontsize=13, fontweight='bold', color=BLUE)
axes[0].set_ylabel('Churn Rate (%)')
axes[0].set_facecolor('#FAFAFA'); axes[0].grid(axis='y', alpha=0.3, linestyle='--')
axes[0].set_ylim(0, 35)

# Gender x Geography
gg = df.groupby(['Geography','Gender'])['Exited'].mean()*100
gg = gg.unstack('Gender').round(1)
x = np.arange(len(gg)); w = 0.35
axes[1].bar(x-w/2, gg['Female'], w, label='Female', color=RED,   edgecolor='white')
axes[1].bar(x+w/2, gg['Male'],   w, label='Male',   color=LIGHT, edgecolor='white')
for i, (f, m) in enumerate(zip(gg['Female'], gg['Male'])):
    axes[1].text(i-w/2, f+0.4, f'{f}%', ha='center', fontsize=9, fontweight='bold')
    axes[1].text(i+w/2, m+0.4, f'{m}%', ha='center', fontsize=9, fontweight='bold')
axes[1].set_xticks(x); axes[1].set_xticklabels(gg.index)
axes[1].set_title('Churn Rate by Country & Gender', fontsize=13, fontweight='bold', color=BLUE)
axes[1].set_ylabel('Churn Rate (%)')
axes[1].legend(); axes[1].set_facecolor('#FAFAFA'); axes[1].grid(axis='y', alpha=0.3, linestyle='--')

plt.suptitle('Gender Segmentation Analysis', fontsize=14, fontweight='bold', color=BLUE)
plt.tight_layout()
plt.show()

print('\nKey Finding: Female customers churn at 25.1% vs 16.5% for males — an 8.6 point gap across all countries')

---
## Step 9: Credit Score & Tenure Segmentation

In [ ]:
cr = df.groupby('CreditBand', observed=True)['Exited'].agg(['count','mean']).reset_index()
cr.columns = ['CreditBand','Customers','ChurnRate']
cr['ChurnRate%'] = (cr['ChurnRate']*100).round(1)

ten = df.groupby('TenureGroup', observed=True)['Exited'].agg(['count','mean']).reset_index()
ten.columns = ['TenureGroup','Customers','ChurnRate']
ten['ChurnRate%'] = (ten['ChurnRate']*100).round(1)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

axes[0].bar(cr['CreditBand'], cr['ChurnRate%'], color=LIGHT, edgecolor='white', width=0.5)
for i, v in enumerate(cr['ChurnRate%']):
    axes[0].text(i, v+0.2, f'{v}%', ha='center', fontsize=12, fontweight='bold')
axes[0].set_title('Churn Rate by Credit Score Band', fontsize=13, fontweight='bold', color=BLUE)
axes[0].set_ylabel('Churn Rate (%)')
axes[0].set_facecolor('#FAFAFA'); axes[0].grid(axis='y', alpha=0.3, linestyle='--')
axes[0].set_ylim(0, 30)

axes[1].bar(ten['TenureGroup'], ten['ChurnRate%'], color=LIGHT, edgecolor='white', width=0.5)
for i, v in enumerate(ten['ChurnRate%']):
    axes[1].text(i, v+0.2, f'{v}%', ha='center', fontsize=12, fontweight='bold')
axes[1].set_title('Churn Rate by Tenure Group', fontsize=13, fontweight='bold', color=BLUE)
axes[1].set_ylabel('Churn Rate (%)')
axes[1].set_facecolor('#FAFAFA'); axes[1].grid(axis='y', alpha=0.3, linestyle='--')
axes[1].set_ylim(0, 30)

plt.suptitle('Credit Score & Tenure Segmentation', fontsize=14, fontweight='bold', color=BLUE)
plt.tight_layout()
plt.show()

print(cr[['CreditBand','Customers','ChurnRate%']])
print()
print(ten[['TenureGroup','Customers','ChurnRate%']])
print('\nKey Finding: Credit score and tenure show minimal variation — they are NOT the key churn drivers')

---
## Step 10: Balance Segmentation & Product Analysis

In [ ]:
bal = df.groupby('BalanceSeg', observed=True)['Exited'].agg(['count','mean']).reset_index()
bal.columns = ['BalanceSeg','Customers','ChurnRate']
bal['ChurnRate%'] = (bal['ChurnRate']*100).round(1)

prod = df.groupby('NumOfProducts')['Exited'].agg(['count','mean']).reset_index()
prod.columns = ['NumOfProducts','Customers','ChurnRate']
prod['ChurnRate%'] = (prod['ChurnRate']*100).round(1)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

colors_bal = [RED if v == bal['ChurnRate%'].max() else LIGHT for v in bal['ChurnRate%']]
axes[0].bar(bal['BalanceSeg'], bal['ChurnRate%'], color=colors_bal, edgecolor='white', width=0.5)
for i, v in enumerate(bal['ChurnRate%']):
    axes[0].text(i, v+0.4, f'{v}%', ha='center', fontsize=12, fontweight='bold')
axes[0].set_title('Churn Rate by Balance Segment', fontsize=13, fontweight='bold', color=BLUE)
axes[0].set_ylabel('Churn Rate (%)')
axes[0].set_facecolor('#FAFAFA'); axes[0].grid(axis='y', alpha=0.3, linestyle='--')
axes[0].set_ylim(0, 45)

prod_colors = [RED if v >= 50 else ('green' if v < 15 else GOLD) for v in prod['ChurnRate%']]
axes[1].bar([f'{p} Product(s)' for p in prod['NumOfProducts']], prod['ChurnRate%'],
            color=prod_colors, edgecolor='white', width=0.6)
for i, v in enumerate(prod['ChurnRate%']):
    axes[1].text(i, v+0.5, f'{v}%', ha='center', fontsize=12, fontweight='bold')
axes[1].set_title('Churn Rate by Number of Products', fontsize=13, fontweight='bold', color=BLUE)
axes[1].set_ylabel('Churn Rate (%)')
axes[1].set_facecolor('#FAFAFA'); axes[1].grid(axis='y', alpha=0.3, linestyle='--')
axes[1].set_ylim(0, 115)

plt.suptitle('Balance & Product Segmentation', fontsize=14, fontweight='bold', color=BLUE)
plt.tight_layout()
plt.show()

print(prod[['NumOfProducts','Customers','ChurnRate%']])
print('\nKey Finding: 2-product customers churn at just 7.6% — the sweet spot')
print('Key Finding: 3-product = 82.7% churn, 4-product = 100% churn. Cross-selling is backfiring!')

---
## Step 11: Member Activity Analysis

In [ ]:
act = df.groupby('IsActiveMember').agg(
    Customers=('Exited','count'),
    Churned=('Exited','sum'),
    ChurnRate=('Exited','mean')
).reset_index()
act['Label']     = act['IsActiveMember'].map({0:'Inactive', 1:'Active'})
act['ChurnRate%'] = (act['ChurnRate']*100).round(1)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

axes[0].bar(act['Label'], act['ChurnRate%'], color=[RED, GREEN], edgecolor='white', width=0.5)
for i, v in enumerate(act['ChurnRate%']):
    axes[0].text(i, v+0.3, f'{v}%', ha='center', fontsize=13, fontweight='bold')
axes[0].set_title('Churn Rate: Active vs Inactive Members', fontsize=13, fontweight='bold', color=BLUE)
axes[0].set_ylabel('Churn Rate (%)')
axes[0].set_facecolor('#FAFAFA'); axes[0].grid(axis='y', alpha=0.3, linestyle='--')
axes[0].set_ylim(0, 35)

# Active x Geography
ag = df.groupby(['Geography','IsActiveMember'])['Exited'].mean()*100
ag = ag.unstack('IsActiveMember').round(1)
ag.columns = ['Inactive','Active']
x = np.arange(len(ag)); w = 0.35
axes[1].bar(x-w/2, ag['Inactive'], w, label='Inactive', color=RED,   edgecolor='white')
axes[1].bar(x+w/2, ag['Active'],   w, label='Active',   color=GREEN, edgecolor='white')
axes[1].set_xticks(x); axes[1].set_xticklabels(ag.index)
axes[1].set_title('Active vs Inactive Churn by Country', fontsize=13, fontweight='bold', color=BLUE)
axes[1].set_ylabel('Churn Rate (%)')
axes[1].legend(); axes[1].set_facecolor('#FAFAFA'); axes[1].grid(axis='y', alpha=0.3, linestyle='--')

plt.suptitle('Member Activity Analysis', fontsize=14, fontweight='bold', color=BLUE)
plt.tight_layout()
plt.show()

print(act[['Label','Customers','Churned','ChurnRate%']])
print('\nKey Finding: Inactive members churn at 26.9% vs 14.3% for active — nearly 2x higher')

---
## Step 12: Cross-Segment Heatmaps

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Geography x Age
pivot1 = df.groupby(['Geography','AgeGroup'], observed=True)['Exited'].mean()*100
pivot1 = pivot1.unstack('AgeGroup').round(1)
sns.heatmap(pivot1, annot=True, fmt='.1f', cmap='RdYlGn_r',
            ax=axes[0], linewidths=0.5, annot_kws={'size':11,'weight':'bold'},
            cbar_kws={'label':'Churn Rate (%)'})
axes[0].set_title('Churn Rate: Geography × Age Group', fontsize=12, fontweight='bold', color=BLUE)
axes[0].set_ylabel('Country'); axes[0].set_xlabel('Age Group')

# Geography x Gender
pivot2 = df.groupby(['Geography','Gender'])['Exited'].mean()*100
pivot2 = pivot2.unstack('Gender').round(1)
sns.heatmap(pivot2, annot=True, fmt='.1f', cmap='RdYlGn_r',
            ax=axes[1], linewidths=0.5, annot_kws={'size':13,'weight':'bold'},
            cbar_kws={'label':'Churn Rate (%)'})
axes[1].set_title('Churn Rate: Geography × Gender', fontsize=12, fontweight='bold', color=BLUE)
axes[1].set_ylabel('Country'); axes[1].set_xlabel('Gender')

plt.suptitle('Cross-Segment Churn Heatmaps', fontsize=14, fontweight='bold', color=BLUE)
plt.tight_layout()
plt.show()

---
## Step 13: High-Value Customer Churn Analysis

In [ ]:
threshold = df['Balance'].quantile(0.75)
hv = df[df['Balance'] >= threshold]
hv_churn  = hv['Exited'].mean()*100
all_churn = df['Exited'].mean()*100
hv_churned_count = hv['Exited'].sum()
avg_bal_churned  = hv[hv['Exited']==1]['Balance'].mean()
total_assets_lost = hv[hv['Exited']==1]['Balance'].sum()

print('=' * 50)
print('     HIGH-VALUE CUSTOMER CHURN ANALYSIS')
print('=' * 50)
print(f'  Balance threshold (top 25%): €{threshold:,.0f}')
print(f'  High-value customers:        {len(hv):,}')
print(f'  Churned from this group:     {hv_churned_count:,}')
print(f'  High-value churn rate:       {hv_churn:.1f}%')
print(f'  Overall churn rate:          {all_churn:.1f}%')
print(f'  Avg balance of HV churners:  €{avg_bal_churned:,.0f}')
print(f'  Total assets lost:           €{total_assets_lost:,.0f}')
print('=' * 50)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

axes[0].bar(['All Customers', 'High-Value (Top 25%)'],
            [round(all_churn,1), round(hv_churn,1)],
            color=[LIGHT, RED], edgecolor='white', width=0.5)
for i, v in enumerate([round(all_churn,1), round(hv_churn,1)]):
    axes[0].text(i, v+0.3, f'{v}%', ha='center', fontsize=13, fontweight='bold')
axes[0].set_title('High-Value vs Overall Churn Rate', fontsize=13, fontweight='bold', color=BLUE)
axes[0].set_ylabel('Churn Rate (%)')
axes[0].set_facecolor('#FAFAFA'); axes[0].grid(axis='y', alpha=0.3, linestyle='--')
axes[0].set_ylim(0, 30)

# Balance distribution
axes[1].hist(df[df['Exited']==0]['Balance'], bins=40, alpha=0.6, color=GREEN, label='Retained', edgecolor='white')
axes[1].hist(df[df['Exited']==1]['Balance'], bins=40, alpha=0.7, color=RED,   label='Churned',  edgecolor='white')
axes[1].axvline(threshold, color=BLUE, linestyle='--', linewidth=2, label=f'Top 25% threshold (€{threshold:,.0f})')
axes[1].set_title('Balance Distribution: Churned vs Retained', fontsize=13, fontweight='bold', color=BLUE)
axes[1].set_xlabel('Account Balance (€)'); axes[1].set_ylabel('Number of Customers')
axes[1].legend(); axes[1].set_facecolor('#FAFAFA'); axes[1].grid(axis='y', alpha=0.3, linestyle='--')

plt.suptitle('High-Value Customer Analysis', fontsize=14, fontweight='bold', color=BLUE)
plt.tight_layout()
plt.show()

---
## Step 14: Key Findings Summary

In [ ]:
findings = {
    'Overall Churn Rate':        '20.4%',
    'Germany Churn Rate':        '32.4% (double France & Spain)',
    'Age 46-60 Churn Rate':      '51.1% (highest segment)',
    'Female Churn Rate':         '25.1% (9pts above males)',
    'Inactive Member Churn':     '26.9% (nearly 2x active)',
    '3-Product Churn Rate':      '82.7%',
    '4-Product Churn Rate':      '100% (every customer exited)',
    '2-Product Churn Rate':      '7.6% (optimal segment)',
    'High-Value Churn Rate':     '23.7% (above overall average)',
    'Credit Score Impact':       'Minimal — not a churn driver',
    'Tenure Impact':             'Minimal — not a churn driver',
}

print('=' * 60)
print('          KEY FINDINGS SUMMARY')
print('=' * 60)
for k, v in findings.items():
    print(f'  {k:<28} : {v}')
print('=' * 60)

---
## Step 15: Recommendations

In [ ]:
recommendations = [
    ('Germany Investigation',        'Churn at 32.4% — double other markets. Conduct exit interviews, audit pricing and service quality immediately.'),
    ('Retention Programme (46-60)',   'This group churns at 51.1%. Assign relationship managers and design loyalty tiers targeting this cohort.'),
    ('Close the Gender Gap',          'Female customers churn 8.6pts above males. Review product design and communication for female segment.'),
    ('Re-engage Inactive Members',    'Trigger personalised outreach at 60-90 days of inactivity before exit decision is made.'),
    ('Fix Cross-Selling Strategy',    '3-4 product customers churn at 83-100%. Audit bundling practices. The 2-product sweet spot must be protected.'),
]

print('=' * 70)
print('                    RECOMMENDATIONS')
print('=' * 70)
for i, (title, detail) in enumerate(recommendations, 1):
    print(f'\n  {i}. {title}')
    print(f'     {detail}')
print('\n' + '=' * 70)
print('  M. Anjali | Financial Analyst Intern')
print('  Unified Mentor × European Central Bank | February 2026')
print('=' * 70)